# U1.C3

During this progress point, players talked with Dr. Toppo and watched her argumentation tutorials and completed two sections of the argumentation. White means not completing this progress; Green means submitting the correct argument at the first attempt; Yellow means submitting the correct argument using more than one attempt.

In [ ]:
pids = [
    "sean3@mhs",
    "wenyi3@mhs",
    "sean1@mhs",
]

### How to Address The Query Below With a Revised Message from the MHS Game

Core Idea: If we include an object called `event_key` in the message from the game, there is almost no calculation/compute time required on the server ... Explained next in the list: 

1. If we make the strings objects in the ordinary JSON sense then we will be able to process this significantly more quickly. 
2. Current Logic: 
    - 34 (quest ID) : This node indicates the student has completed the progress point. This node will default the color to white if false. Color can still change based on subsequent criteria. 
    - (these are dialogue and nodeIDs) 33 and 25 or [70/33] or [70/25] : These node IDs indicate successful completion of the progress point. These nodes are only accessed if students make several attempts to complete the corresponding quest. Then Yellow
    - Otherwise Green 
3. Goal Logic (How the game would instantiate and modify an `event_key`): 
    - Ideal: Add an additional variable to the API, such as `event_key` which will indicate whether it is a "question finish event". If this key is not NULL then we know with that piece of data whether the color is white, yellow, or green. 
    - at 34: If FALSE `event_key` is white
    - at 33/25, 70/33 and 70/25: If either of the pairs are true, `event_key` is yellow. 
    - otherwise, `event_key` is green. 

**Sean thought: if the `event_data` includes an `event_key` that is passed as an object instead of a string that needs to be parsed (regex), then that may be sufficient** Could also possibly be set elsewhere. Sean is trying to consider the various trade offs of where the logic lives. 

#### Game Side Possibilities (As Wenyi and Sean are Considering Them)
1. Include `event_key` with the dialogue node event AND
2. the `dialouge_node_event` AND 
3. the `quest_finish_event` 

**CODING STANDARD NOTE:** It is better to use object names like `eventKey` than `event_key`. 

#### Query


In [ ]:
def color_u1c3_for_pid(coll, pid):
    
    has_q34 = coll.count_documents({
        "player_id": pid,
        "event_type": "questEvent",
        "$and": [
            {"event_data": {"$regex": r'"questEventType"\s*:\s*"questActiveEvent"'}},
            {"event_data": {"$regex": r'"questID"\s*:\s*"34"'}}
        ]
    }) > 0

    if not has_q34:
        return "white"

    has_node_33_or_25 = coll.count_documents({
        "player_id": pid,
        "event_type": "DialogueEvent",
        "$and": [
            {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
            {"event_data": {"$regex": r'"conversationId"\s*:\s*"70"'}},
            {"event_data": {"$regex": r'"nodeId"\s*:\s*"(33|25)"'}}
        ]
    }) > 0

    return "yellow" if has_node_33_or_25 else "green"

In [ ]:
pids_u1c3_to_color = {pid: color_u1c3_for_pid(coll, pid) for pid in pids}
pids_u1c3_to_color

{'sean3@mhs': 'green', 'wenyi3@mhs': 'yellow', 'sean1@mhs': 'white'}